In [ ]:
import sys
import os
import random

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from matplotlib import pyplot as plt

import tensorflow as tf
import keras
from keras import backend as K
from keras import initializers
from keras.layers import Dense, Input, Activation, multiply, Lambda
from keras.models import Sequential, Model, load_model
from keras.layers.merge import add, concatenate

from deap import base, creator, tools, algorithms
from multiprocessing import Pool
from scoop import futures

import pickle

In [ ]:
# List of all potential activation functions (Very problem specific)
act_dict = {0: 'linear', 1: 'multiply', 2: 'inverse', 3: 'squared', 4: 'sqrt', 5: 'cubed'}

np.random.seed(1000)
weight_dict = {0: 0, 1: 1, 2: np.random.uniform(0,0.001,1)[0]}
bias_dict = {0: 0, 1: np.random.uniform(0,0.001,1)[0]}
#bias_dict = {0: 0, 1: 1.5}
print(weight_dict)
print(bias_dict)

# Number of layers in the model (Includes input layer)
nlayers = 4

# Number of nodes per layer (Includes input layer)
nNodes = [5, 5, 3, 1]

# Number of variable weight/bias/activation terms to optimize
nact_terms = sum(nNodes[1:])
nweight_terms = sum([nNodes[i-1]*nNodes[i] for i in range(1, nlayers)])
nbias_terms = nact_terms

# Variable to load an old population as the starting population in the genetic algorithm (pickle file)
Load_Old_Population = False

# Parsinomy Value
p = 0.1

In [ ]:
# Empty lists to record what individuals have already been evaluated 
Individual_DB = []
MSE_Test_DB = []

In [ ]:
# Training/Testing Data
df = pd.read_csv("PropellantData_v3.csv")
#print (df.shape)

In [ ]:
# Model inputs
inputs = np.array(df[['a','b', 'c', 'd', 'HeatofFormation']])
inputs = inputs.astype(np.float)

# Model output
outputs = np.array(df['Isp']).reshape(-1, 1)

inputs = np.around(inputs, decimals=6)
outputs = np.around(outputs, decimals=2)

In [ ]:
# Custom square activation function
def squared_act(x):
    return x*x

In [ ]:
# Custom cubed activation function
def cubed_act(x):
    return x**3

In [ ]:
# Custom square root activation function (set up to handle negative inputs and train through them)
def sqrt_act(x):
    return tf.sign(x)* tf.sqrt(tf.abs(x))

In [ ]:
# Custom inverse activation function
def inverse_act(x):
    return (1/x)

In [ ]:
# Custom multiply activation
# Only includes non-zero weighted inputs to avoid only fully connected nodes returning non-zero outputs
# x_inputs - description of node [A, w, w, w, ....., b] ex. [1, 2, 1, 2, 0, ..., 1]
def custom_multiply(x, x_inputs):
    # Lists to split the inputs tensors
    activation_inputs = []
    zero_inputs = []
    
    # splits the inputs tensors into zero tensor (zero incoming weight) and non zero tensor
    for t in range(0, len(x)):
        if x_inputs[t] != 0:
            activation_inputs.append(x[t])
        elif (t == len(x)-1) & (x_inputs[len(x)] == 1):
            activation_inputs.append(x[t])
        else:
            zero_inputs.append(x[t])

    # Checks if list is empty
    if activation_inputs == []:
        activation_tensor = 0
    elif len(activation_inputs) == 1:
        activation_tensor = activation_inputs[0]
    else:
        activation_tensor = multiply(activation_inputs)
    
    # Checks if list is empty
    if zero_inputs == []:
        zero_tensor = 0
    elif len(zero_inputs) == 1:
        zero_tensor = zero_inputs[0]
    else:
        zero_tensor = multiply(zero_inputs)
        
    # Checks if either list contains all the tensors
    if not tf.is_tensor(zero_tensor):
        return activation_tensor
    elif not tf.is_tensor(activation_tensor):
        return zero_tensor
    else:
        # Since all input tensors must be connected to the output, the zero tensors are added to the non zero tensors.
        return add([activation_tensor, zero_tensor])

In [ ]:
# Custom keras layers
# Allows for bias and weight to be set to trainable and non trainable independently
# Normal keras dence layers "trainable" term controls both the bias and the weight
class CustomDense(keras.layers.Layer):
    def __init__(self, num_units, input_num, activation, name, trainable_weight, trainable_bias):
        super(CustomDense, self).__init__()
        self.num_units = num_units
        self.activation = Activation(activation)
        self.trainable_weight = trainable_weight
        self.trainable_bias = trainable_bias
        self.name = name
        name_w = 'w'+self.name[1:]
        name_b = 'b'+self.name[1:]
        self.weight = self.add_weight(shape=(input_num, self.num_units), name=name_w, trainable=self.trainable_weight, initializer="zeros")
        self.bias = self.add_weight(shape=(self.num_units,), name=name_b, trainable=self.trainable_bias, initializer="zeros")
        
    def call(self, input):
        y = tf.matmul(input, self.weight) + self.bias
        y = self.activation(y)
        return y

In [ ]:
# Function to build the nodes within the network
def create_node(inputs, name, trainable, x_input, constraints):
    base = name
    act = act_dict[x_input[0]]
  
    # Collects inputs to the node and sets the connection to the appropriate trainable/bias setting
    # Only the last connection to the node gets a bias (Every connection getting a bias is not useful)
    an = []
    n = []
    for i in range(len(inputs)):
        n = base + str(i + 1)
        if i < len(inputs)-1:
            an.append(CustomDense(1, 1, activation = 'linear', name=n, trainable_weight=trainable[i], trainable_bias=False) (inputs[i]))
        else:
            an.append(CustomDense(1, 1, activation = 'linear', name=n, trainable_weight=trainable[i], trainable_bias=trainable[len(trainable)-1]) (inputs[i]))

    # Apply activation function to the list of inputs
    if (act == "multiply"):
        an = Activation(lambda x: custom_multiply(x, x_inputs=x_input[1:])) (an)
    else:
        an = add(an)
        if (act == "squared"):
            an = Activation(squared_act) (an)
        elif (act == "cubed"):
            an = Activation(cubed_act) (an)
        elif (act == "sqrt"):
            an = Activation(sqrt_act) (an)
        elif (act == "inverse"):
            an = Activation(inverse_act) (an)
        else:
            an = Activation(act) (an)
            
    # Checks for even root functions and connects output to constraint output node
    if act == 'sqrt' or act == '4th_rt':
        constraints.append(Activation('relu')(Lambda(lambda x: tf.negative(x))(an)))       
    return an

In [ ]:
def create_model(x, constraint_strength=10^3):
    #print("Individual: ", x)
    constraints = []

    # Collects a list of the trainable parameters in the model
    trainable_list = []
    for l in range(1, nlayers):
        trainable_list.append([])
        for n in range(nNodes[l]):
            trainable_list[l-1].append([])
            for i in range(1, nNodes[l-1]+1):
                if (x[l-1][n][i] == 2):
                    trainable_list[l-1][n].append(True)
                else:
                    trainable_list[l-1][n].append(False)
            if (x[l-1][n][i+1] == 1):
                trainable_list[l-1][n].append(True)
            else:
                trainable_list[l-1][n].append(False)
    
    inputs = []
    for i in range(nNodes[0]):
        inputs.append(Input(shape=(1,)))
    
    # Builds list of customdense objects and connects each node to the previous layer
    a = []
    for l in range(1, nlayers):
        a.append([])
        for n in range(1, nNodes[l]+1):
            a[l-1].append([])
            if l == 1:
                a[l-1][n-1] = create_node(inputs, 'a' + str(l) + str(n), trainable_list[l-1][n-1], x[l-1][n-1], constraints)
            else:
                a[l-1][n-1] = create_node(a[l-2], 'a' + str(l) + str(n), trainable_list[l-1][n-1], x[l-1][n-1], constraints)
    
    
    # Make some adaptations depending on how many even root activation functions are used
    if len(constraints) > 1:
        constraint = add(constraints)
    elif len(constraints) == 1:
        constraint = constraints[0]
    else:
        constraint = Lambda(lambda x: x-x)(a[0][0])
    
    # Append constraint node
    a[-1].append(constraint)
    model = Model(inputs=inputs, outputs=a[-1])
    
    # Learning rate decay/schedule
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-1,
                                                                decay_steps=50000,
                                                                decay_rate=0.1,
                                                                staircase=False)
    
    # Optimizers
    #optimizer = tf.keras.optimizers.SGD(lr=1e-2)
    #optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    #optimizer = tf.keras.optimizers.Adadelta(learning_rate=lr_schedule)
    #optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr_schedule)
    #optimizer = tf.keras.optimizers.Adamax(learning_rate=lr_schedule)
    #optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-3)
    #optimizer = tf.keras.optimizers.Ftrl(lr=1e-3)
    
    
    model.compile(loss=['mse','mse'], loss_weights=[1, constraint_strength], optimizer=optimizer)
    model.layers.sort(key = lambda x: x.name, reverse=False)
    
    layer_list = []
    for i in range(len(model.layers)):
        name = model.layers[i].name
        if ( ("activation" in name) or ("input" in name) or ("add" in name) or ("multiply" in name) or ("lambda" in name)):
            continue
        else:
            layer_list.append(i)
    
    # Assign weights to the model
    index = 0
    for l in range(1, nlayers):
        for n in range(1, nNodes[l]+1):
            for i in range(1, nNodes[l-1]+1):
                if ((i+1)%(nNodes[l-1]+1)==0):
                    if (model.layers[layer_list[index]].get_weights()[0].shape==(1,1)):
                        model.layers[layer_list[index]].set_weights( [ np.array( [[ weight_dict[x[l-1][n-1][i]] ]] ),  np.array( [ bias_dict[x[l-1][n-1][i+1]] ] ) ] )
                    else:
                        model.layers[layer_list[index]].set_weights( [ np.array( [ bias_dict[x[l-1][n-1][i+1]] ] ),  np.array( [[ weight_dict[x[l-1][n-1][i]] ]] ) ] )
                else:
                    if (model.layers[layer_list[index]].get_weights()[0].shape==(1,1)):
                        model.layers[layer_list[index]].set_weights( [ np.array( [[ weight_dict[x[l-1][n-1][i]] ]] ),  np.array( [ 0 ] ) ] )
                    else:
                        model.layers[layer_list[index]].set_weights( [ np.array( [ 0 ] ),  np.array( [[ weight_dict[x[l-1][n-1][i]] ]] ) ] )
                index += 1

    return model, trainable_list

In [ ]:
class ValidLossNaN(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if np.isnan(logs.get('loss')):
            self.model.stop_training=True

losses = []
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + ' Loss: ' + str(logs.get('loss')) + '                     \n')
        losses.append(logs.get('loss'))

def train(model, train_inputs, train_outputs, verbose=False):
    mae_es= keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                          min_delta=1e-8, verbose=0, mode='auto', restore_best_weights=True)
    
    terminate = keras.callbacks.TerminateOnNaN()

    EPOCHS = 50000 # Number of EPOCHS
    history = model.fit([train_inputs[:, i] for i in range(0, nNodes[0])], [train_outputs, np.zeros(train_outputs.shape)], epochs=EPOCHS,
                        shuffle=False, batch_size=32, verbose = 0, callbacks=[ValidLossNaN(), terminate, mae_es],
                        validation_split=0.3)
    # Test changing batch size
    if verbose:
        plt.figure()
        plt.xlabel('Epoch')
        plt.ylabel('Mean Sq Error')
        plt.plot(history.epoch, np.array(history.history['loss']),label='Training loss')
        plt.plot(history.epoch, np.array(history.history['val_loss']),label='Val loss')
        plt.legend()
        plt.show()
    return history

In [ ]:
# KFolds function to evaluate the model
def cv_error(individual, inputs, outputs):
    # Get splits for test train
    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    kf.get_n_splits(inputs)
    
    cv_mse_list = []
    

    for train_index, test_index in kf.split(inputs):
        # Create a model with initial weights
        # Weights are not saved across each split
        new_model, trainable = create_model(individual)
        
        # Train model
        if (any(trainable) == True):
            try:
                trained = True
                train(new_model, inputs[train_index, :], outputs[train_index], verbose=0)
            except:
                trained = False
                new_model, trainable = create_model(individual)
    
        # Get weights/biases
        weights = new_model.get_weights()
        weight_list = []
        bias_list = []
        for weight in weights:
            if (weight.shape == (1,1)):
                weight_list.append(weight[0])
            else:
                bias_list.append(weight[0])
        weight_list = np.array(weight_list)
        bias_list = np.array(bias_list)
    
        #handle nan weights and biases
        if (np.isnan(weight_list).any()):
            cv_mse = 1e50
        elif (np.isnan(bias_list).any()):
            cv_mse = 1e50
        elif not trained:
            cv_mse = 1e50
        else:
            cv_mse = new_model.evaluate([inputs[test_index, i] for i in range(0, nNodes[0])], [outputs[test_index], np.zeros(outputs[test_index].shape)], verbose=0)
            if (np.isnan(cv_mse).any()):
                cv_mse = 1e50
            else:
                cv_mse = cv_mse[1]
                cv_mse = np.around(cv_mse, decimals=6)
        cv_mse_list.append(cv_mse)
    return np.mean(cv_mse_list)

In [ ]:
def f3(w):
    return w

In [ ]:
def individual_reader(temp):
    act_terms = temp[:nact_terms]
    weight_terms = temp[nact_terms:nact_terms + nweight_terms]
    bias_terms = temp[nact_terms + nweight_terms:]
    print(weight_terms)
    x = []
    for l in range(1, nlayers):
        x.append([])
        for n in range(1, nNodes[l]+1):
            x[l-1].append([])
            x[l-1][n-1].append(act_terms.pop(0))
            for i in range(1, nNodes[l-1]+1):
                x[l-1][n-1].append(weight_terms.pop(0))
            x[l-1][n-1].append(bias_terms.pop(0))
    return x

In [ ]:
def objective_function(individual):
    gen = individual.pop()
    
    if individual not in Individual_DB:
        # Evaluate model
        mse_test = cv_error(individual, inputs, outputs)
        
        # Determine model complexity
        actfunc_term = 0
        wtbs_term = 0
        for l in range(1, nlayers):
            for n in range(nNodes[l]):
                actfunc_term += individual[l-1][n][0]
                wtbs_term += np.sum(individual[l-1][n][1:nNodes[l-1]+1])
                wtbs_term += individual[l-1][n][nNodes[l-1]+1]*2
                
        # Main Objective function for GA    
        obj = mse_test + p*(np.sum(actfunc_term) + wtbs_term)
        #obj = mse_test_term
        print ("Individual: ", individual, flush=True)
        print ("Objective function: ", mse_test, np.sum(actfunc_term), wtbs_term, obj, gen, flush=True)
        
        Individual_DB.append(individual)
        MSE_Test_DB.append(mse_test)
        
    else:
        # Evaluate model
        mse_test = MSE_Test_DB[Individual_DB.index(individual)]
        
        # Determine model complexity
        actfunc_term = 0
        wtbs_term = 0
        for l in range(1, nlayers):
            for n in range(nNodes[l]):
                actfunc_term += individual[l-1][n][0]
                wtbs_term += np.sum(individual[l-1][n][1:nNodes[l-1]+1])
                wtbs_term += individual[l-1][n][nNodes[l-1]+1]*2
                
        # Main Objective function for GA
        obj = mse_test + p*(np.sum(actfunc_term) + wtbs_term)
    
    K.clear_session()
    #tf.reset_default_graph()
    return (obj,)

In [ ]:
# Function to create random individuals
def custom_initRepeat(container, func, max1, max2, max3):
    x = []
    for l in range(1, nlayers):
        x.append([])
        for n in range(1, nNodes[l]+1):
            x[l-1].append([])
            x[l-1][n-1].append(func(0, max1))
            for i in range(1, nNodes[l-1]+1):
                x[l-1][n-1].append(func(0, max2))
            x[l-1][n-1].append(func(0, max3))
    return container(x)

In [ ]:
# Custom GA mutation. Only mutate each activation, weight, bias to a valid setting
def custom_mutation(individual, indpb, max1, max2, max3):
    for l in range(1, nlayers):
        for n in range(1, nNodes[l]+1):
            if random.random() < indpb:
                individual[l-1][n-1][0] = random.randint(0, max1)
            for i in range(1, nNodes[l-1]+1):
                if random.random() < indpb:
                    individual[l-1][n-1][i] = random.randint(0, max2)
            if random.random() < indpb:
                individual[l-1][n-1][i+1] = random.randint(0, max3)
    return individual,

In [ ]:
# Custom GA crossover. Switches nodes from the same location on each network
def custom_crossover(individual1, individual2):
    LAYER = random.randint(1, nlayers-1)
    NODE = random.randint(0, nNodes[LAYER]-1)
    temp = individual1[LAYER-1][NODE]
    individual1[LAYER-1][NODE] = individual2[LAYER-1][NODE]
    individual2[LAYER-1][NODE] = temp
    return individual1, individual2

In [ ]:
def custom_eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm reproduce the simplest evolutionary algorithm as
    presented in chapter 7 of [Back2000]_.

    :param population: A list of individuals.
    :param toolbox: A :class:`~deap.base.Toolbox` that contains the evolution
                    operators.
    :param cxpb: The probability of mating two individuals.
    :param mutpb: The probability of mutating an individual.
    :param ngen: The number of generation.
    :param stats: A :class:`~deap.tools.Statistics` object that is updated
                  inplace, optional.
    :param halloffame: A :class:`~deap.tools.HallOfFame` object that will
                       contain the best individuals, optional.
    :param verbose: Whether or not to log the statistics.
    :returns: The final population
    :returns: A class:`~deap.tools.Logbook` with the statistics of the
              evolution

    The algorithm takes in a population and evolves it in place using the
    :meth:`varAnd` method. It returns the optimized population and a
    :class:`~deap.tools.Logbook` with the statistics of the evolution. The
    logbook will contain the generation number, the number of evaluations for
    each generation and the statistics if a :class:`~deap.tools.Statistics` is
    given as argument. The *cxpb* and *mutpb* arguments are passed to the
    :func:`varAnd` function. The pseudocode goes as follow ::

        evaluate(population)
        for g in range(ngen):
            population = select(population, len(population))
            offspring = varAnd(population, toolbox, cxpb, mutpb)
            evaluate(offspring)
            population = offspring

    As stated in the pseudocode above, the algorithm goes as follow. First, it
    evaluates the individuals with an invalid fitness. Second, it enters the
    generational loop where the selection procedure is applied to entirely
    replace the parental population. The 1:1 replacement ratio of this
    algorithm **requires** the selection procedure to be stochastic and to
    select multiple times the same individual, for example,
    :func:`~deap.tools.selTournament` and :func:`~deap.tools.selRoulette`.
    Third, it applies the :func:`varAnd` function to produce the next
    generation population. Fourth, it evaluates the new individuals and
    compute the statistics on this population. Finally, when *ngen*
    generations are done, the algorithm returns a tuple with the final
    population and a :class:`~deap.tools.Logbook` of the evolution.

    .. note::

        Using a non-stochastic selection method will result in no selection as
        the operator selects *n* individuals from a pool of *n*.

    This function expects the :meth:`toolbox.mate`, :meth:`toolbox.mutate`,
    :meth:`toolbox.select` and :meth:`toolbox.evaluate` aliases to be
    registered in the toolbox.

    .. [Back2000] Back, Fogel and Michalewicz, "Evolutionary Computation 1 :
       Basic Algorithms and Operators", 2000.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    for ind in invalid_ind:
        ind.append(0)
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = custom_varAnd(offspring, toolbox, cxpb, mutpb, gen, ngen)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid_ind:
            ind.append(gen)
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            if len(ind) > (nlayers-1):
                ind = ind.pop()

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

In [ ]:
def custom_varAnd(population, toolbox, cxpb, mutpb, gen, ngen):
    """Part of an evolutionary algorithm applying only the variation part
    (crossover **and** mutation). The modified individuals have their
    fitness invalidated. The individuals are cloned so returned population is
    independent of the input population.

    :param population: A list of individuals to vary.
    :param toolbox: A :class:`~deap.base.Toolbox` that contains the evolution
                    operators.
    :param cxpb: The probability of mating two individuals.
    :param mutpb: The probability of mutating an individual.
    :returns: A list of varied individuals that are independent of their
              parents.

    The variation goes as follow. First, the parental population
    :math:`P_\mathrm{p}` is duplicated using the :meth:`toolbox.clone` method
    and the result is put into the offspring population :math:`P_\mathrm{o}`.  A
    first loop over :math:`P_\mathrm{o}` is executed to mate pairs of
    consecutive individuals. According to the crossover probability *cxpb*, the
    individuals :math:`\mathbf{x}_i` and :math:`\mathbf{x}_{i+1}` are mated
    using the :meth:`toolbox.mate` method. The resulting children
    :math:`\mathbf{y}_i` and :math:`\mathbf{y}_{i+1}` replace their respective
    parents in :math:`P_\mathrm{o}`. A second loop over the resulting
    :math:`P_\mathrm{o}` is executed to mutate every individual with a
    probability *mutpb*. When an individual is mutated it replaces its not
    mutated version in :math:`P_\mathrm{o}`. The resulting :math:`P_\mathrm{o}`
    is returned.

    This variation is named *And* because of its propensity to apply both
    crossover and mutation on the individuals. Note that both operators are
    not applied systematically, the resulting individuals can be generated from
    crossover only, mutation only, crossover and mutation, and reproduction
    according to the given probabilities. Both probabilities should be in
    :math:`[0, 1]`.
    """
    offspring = [toolbox.clone(ind) for ind in population]

    # Apply crossover and mutation on the offspring
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                          offspring[i])
            del offspring[i - 1].fitness.values, offspring[i].fitness.values

    for i in range(len(offspring)):
        if random.random() < mutpb[(gen-1)//(ngen//len(mutpb))]:
            offspring[i], = toolbox.mutate(offspring[i], mutpb[(gen-1)//(ngen//len(mutpb))])
            del offspring[i].fitness.values
            
    return offspring

In [ ]:
################### DEAP #####################
#create fitness class and individual class
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
#pool = Pool(1)
toolbox.register("map", futures.map)
#toolbox.register("attr_int", random.randint, 0, 3)

#gen = initRepeat(list, random.randint, 3, 7, 4)
toolbox.register("create_individual", custom_initRepeat, creator.Individual, random.randint, 
                 max1=5, max2=2, max3=1)
toolbox.register("population", tools.initRepeat, list, toolbox.create_individual)

# Crossover probability
cxpb = 0.5
# Variable mutation (decreases across generations)
# Length of mutpb list must divide evenly into ngens
mutpb = [0.9, 0.8, 0.5, 0.2, 0.1]
if Load_Old_Population:
    mutpb = [0.1, 0.1, 0.1, 0.1, 0.1]

# Number of generations
ngens = 5

toolbox.register("mate", custom_crossover)
#toolbox.register("mutate", tools.mutUniformInt, low=0, up=3, indpb=mutpb)
toolbox.register("mutate", custom_mutation, max1=3, max2=2, max3=1)
toolbox.register("select", tools.selTournament, tournsize=80)
toolbox.register("evaluate", objective_function)

def main():
    random.seed(10000)
    
    if not Load_Old_Population:
        population = toolbox.population(n=5)
    else:
        with open("P" + str(p) + "_1", 'rb') as f:
            population = pickle.load(f)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    pop, logbook = custom_eaSimple(population, toolbox, cxpb, mutpb, ngens, stats=stats, halloffame=hof, verbose=True)

    return pop, logbook, hof

if __name__ == "__main__":
    pop, logbook, hof = main()
    
    # Save final population
    filename = "P" + str(p)
    with open(filename, 'wb') as f:
        pickle.dump(pop, f)
        f.close()
        
    print (logbook, flush=True)
    print (hof, flush=True)
